<div style="display: flex; align-items: center; padding: 20px; background-color: #f0f2f6; border-radius: 10px; border: 2px solid #007bff;">
    <img src="../logo.png" style="width: 80px; height: auto; margin-right: 20px;">
    <div style="flex: 1; text-align: left;">
    <h1 style="color: #007bff; margin-bottom: 5px;">GLY 6739.017S26: Computational Seismology</h1>
    <h3 style="color: #666;">Notebook 09: Analog-to-Digital Converter demo</h3>
    <p style="color: red;"><i>Glenn Thompson | Spring 2026</i></p>
    </div>
</div>


In this notebook, we simulate the process of analog-to-digital conversion (ADC). This is how real world signals are converted to numbers on a computer. Think of air temperature, for example. It changes continuously, effectively at infinite samples per second. And it is a real number - it has an infinite number of levels (it is not "quantized"). We are limited only by our ability to sample it, and measure it precisely (and accurately). Remember: numbers on a computer are approximations. So real signals - including air pressure signals, and ground-motion (seismic) signals - can only be approximated on a computer. ADC is the process of how we do that.

But wait - we are doing this on a computer. So we will approximate an analog signal by using a very high sampling rate (10 kHz), and using 64-bit float, which gives us 2^64 different signal levels - close enough to infinity for our purposes.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from ipywidgets import interact, IntSlider, FloatSlider, Dropdown, VBox, HTML, Checkbox
from IPython.display import display

# -------------------------
# Globals / time axis
# -------------------------
FS_ANALOG = 100_000.0   # Hz (higher helps interpolation feel "continuous")
T = 6.0              # seconds (long enough to see low fs behaviour)
t_analog = np.arange(0, T, 1/FS_ANALOG)

# -------------------------
# Signal generators
# -------------------------
def make_signal(signal_type, analog_amp, seed=0):
    """
    Return an "analog" waveform x(t) (float64) defined on t_analog.
    We normalize each base signal to max abs ~1, then scale by analog_amp.
    """
    t = t_analog
    rng = np.random.default_rng(seed)

    if signal_type == "Sum of sines":
        x = (
            0.85*np.sin(2*np.pi*1.0*t) +
            0.20*np.sin(2*np.pi*7.0*t + 0.4) +
            0.08*np.sin(2*np.pi*15.0*t + 1.2)
        )

    elif signal_type == "Sawtooth":
        # sawtooth at 2 Hz (range -1..1)
        f = 2.0
        frac = (t * f) % 1.0
        x = 2.0*frac - 1.0
        # add a bit of band-limited-ish wobble so spectrum isn't purely harmonic
        x += 0.08*np.sin(2*np.pi*20.0*t)

    elif signal_type == "Earthquake-like (burst + noise)":
        # A simple “event” model: noise + a damped sinusoid burst (like a ringy arrival/coda)
        noise = 0.15 * rng.standard_normal(len(t))

        t0 = 2.0          # event start (s)
        f0 = 6.0          # Hz dominant
        decay = 1.2       # decay constant
        envelope = np.exp(-(t - t0) * decay) * (t >= t0)
        burst = envelope * np.sin(2*np.pi*f0*(t - t0))

        # Add a slightly higher-frequency component to show aliasing when fs is low
        burst += 0.35 * envelope * np.sin(2*np.pi*18.0*(t - t0) + 0.3)

        x = noise + burst

    else:
        raise ValueError("Unknown signal type.")

    # Normalize then scale
    x = x / (np.max(np.abs(x)) + 1e-12)
    return analog_amp * x

# -------------------------
# ADC helpers
# -------------------------
def quant_levels(bits, vref):
    L = 2**bits
    return np.linspace(-vref, vref, L)

def quantize_uniform(x, bits, vref):
    """
    Uniform quantizer for x in [-vref, +vref].
    Returns quantized x, step size q, and clipped mask.
    """
    clipped = (x < -vref) | (x > vref)
    x_clip = np.clip(x, -vref, vref)

    L = 2**bits
    q = (2*vref) / (L - 1)
    xq = np.round((x_clip + vref) / q) * q - vref
    return xq, q, clipped, x_clip

def sample_via_interpolation(fs_sample, x_analog):
    """
    Sample analog waveform at arbitrary fs_sample using linear interpolation.
    """
    ts = np.arange(0, T, 1/fs_sample)
    xs = np.interp(ts, t_analog, x_analog)
    return ts, xs

def amp_spectrum(x, fs):
    """
    One-sided amplitude spectrum (magnitude) with Hann window.
    """
    x = np.asarray(x)
    if len(x) < 8:
        return np.array([0.0]), np.array([0.0])
    w = np.hanning(len(x))
    X = np.fft.rfft(x * w)
    f = np.fft.rfftfreq(len(x), d=1/fs)
    mag = np.abs(X)
    return f, mag

# -------------------------
# Interactive view
# -------------------------
def adc_view(
    signal="Sum of sines",
    bits=8,
    fs=50,
    analog_amp=0.8,
    adc_fullscale=1.0,
    show_all_levels=False,
    seed=0
):
    # Build analog signal
    x_analog = make_signal(signal, analog_amp=analog_amp, seed=seed)

    # Sample at arbitrary fs (interpolation)
    ts, xs = sample_via_interpolation(fs_sample=float(fs), x_analog=x_analog)

    # Quantize sampled points (with clipping)
    xq, q, clipped_mask, xs_clipped = quantize_uniform(xs, bits=bits, vref=adc_fullscale)

    # Quantization levels for horizontal lines
    levels = quant_levels(bits, vref=adc_fullscale)
    L = len(levels)

    # Decide how many horizontal lines to draw
    if show_all_levels:
        # This can be VERY heavy for bits=12; use with caution.
        levels_to_draw = levels
        level_note = f"(drawing all {L} levels)"
    else:
        max_lines = 128
        if L <= max_lines:
            levels_to_draw = levels
            level_note = f"(drawing all {L} levels)"
        else:
            k = int(np.ceil(L / max_lines))
            levels_to_draw = levels[::k]
            level_note = f"(levels={L}; drawing every {k}-th → {len(levels_to_draw)} lines)"

    # Clipping stats
    nclip = int(np.sum(clipped_mask))
    clip_note = f"Clipping: {nclip}/{len(xs)} sampled points clipped" if nclip > 0 else "Clipping: none"

    # -------------------------
    # Plot 1: Time series with quantization levels + sampled/quantized dots
    # -------------------------
    plt.figure(figsize=(11, 5))

    # Analog reference
    plt.plot(t_analog, x_analog, linewidth=1, label=f"Analog reference (fs={FS_ANALOG:.0f} Hz)")

    # ADC full-scale lines
    plt.axhline(+adc_fullscale, linewidth=1.2, alpha=0.6)
    plt.axhline(-adc_fullscale, linewidth=1.2, alpha=0.6)

    # Quantization level lines
    for y in levels_to_draw:
        plt.axhline(y, linewidth=0.6, alpha=0.25)

    # Sampled (pre-clip) points (faint)
    plt.plot(ts, xs, marker='o', linestyle='None', markersize=5, alpha=0.25, label="Sampled (pre-clip)")

    # Mark samples outside ADC range
    if nclip > 0:
        plt.plot(ts[clipped_mask], xs[clipped_mask], marker='x', linestyle='None',
                 markersize=7, alpha=0.9, label="Samples outside ADC range")

    # Quantized points (strong)
    plt.plot(ts, xq, marker='o', linestyle='None', markersize=7, label="Sampled + quantized")

    # Step line for sample-and-hold feel
    plt.step(ts, xq, where='mid', linewidth=1, alpha=0.8)

    plt.title(
        f"ADC simulator — signal='{signal}', fs={fs} Hz, bits={bits} (Δ={q:.4g}), "
        f"analog_amp={analog_amp:.2f}, fullscale=±{adc_fullscale:.2f}  {level_note}"
    )
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude (arb.)")
    ypad = max(0.15, 0.15 * adc_fullscale)
    plt.ylim(-adc_fullscale - ypad, adc_fullscale + ypad)
    plt.grid(True, alpha=0.15)
    plt.legend(loc="upper right")
    plt.show()

    # -------------------------
    # Plot 2: Amplitude spectrum (analog vs sampled+quantized)
    # -------------------------
    fA, mA = amp_spectrum(x_analog, FS_ANALOG)
    fQ, mQ = amp_spectrum(xq, float(fs))

    # To make spectra comparable visually, we can normalize each by its max (optional but helpful)
    mA_n = mA / (mA.max() + 1e-12)
    mQ_n = mQ / (mQ.max() + 1e-12)

    plt.figure(figsize=(11, 4))
    #plt.semilogy(fA, mA_n, '-', linewidth=3, label="Analog spectrum (normalized)")
    plt.semilogy(fA, mA_n, '-',  label="Analog spectrum (normalized)")
    #plt.semilogy(fQ, mQ_n, '-', linewidth=2, label="Sampled+quantized spectrum (normalized)")
    plt.semilogy(fQ, mQ_n, marker='o', linestyle='None', markersize=7, label="Sampled+quantized spectrum (normalized)")
    plt.title("Amplitude spectrum comparison (watch aliasing + quantization noise)")
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Normalized magnitude")
    #plt.xlim(0, min(200.0, FS_ANALOG/2))  # keep it readable; adjust as desired
    plt.xlim(0, min(fs/2, FS_ANALOG/2))
    plt.grid(True, alpha=0.15)
    plt.legend(loc="upper right")
    plt.show()

    # Console notes
    print(f"Sampling: interpolated analog (fs={FS_ANALOG:.0f} Hz) to fs={fs} Hz → {len(ts)} samples over {T:.1f} s.")
    print(f"Quantization: {bits}-bit uniform ADC across ±{adc_fullscale} → {2**bits} levels, step Δ={q:.6g}.")
    print(clip_note)
    if bits >= 10 and not show_all_levels:
        print("Note: high bit depths have many levels; the plot shows a downsampled set of level lines for clarity.")

# -------------------------
# Widgets UI
# -------------------------
ui = VBox([
    HTML(
        "<b>Interactive ADC demo (interpolated sampling + selectable signals + spectrum)</b><br>"
        "Choose <b>signal type</b>, <b>bits</b>, <b>sampling rate</b>, <b>analog amplitude</b>, and <b>ADC full-scale</b>.<br>"
        "Time plot: analog waveform + quantization levels + sampled points + quantized points (sample-and-hold).<br>"
        "Spectrum plot: analog vs sampled+quantized (normalized)."
    )
])
display(ui)

interact(
    adc_view,
    signal=Dropdown(
        options=["Sum of sines", "Earthquake-like (burst + noise)", "Sawtooth"],
        value="Sum of sines",
        description="signal"
    ),
    bits=IntSlider(value=8, min=1, max=12, step=1, description="bits"),
    fs=IntSlider(value=50, min=1, max=500, step=1, description="fs (Hz)"),
    analog_amp=FloatSlider(value=0.8, min=0.1, max=2.0, step=0.05, description="analog_amp"),
    adc_fullscale=FloatSlider(value=1.0, min=0.2, max=2.0, step=0.05, description="fullscale"),
    show_all_levels=Checkbox(value=False, description="draw all levels"),
    seed=IntSlider(value=0, min=0, max=20, step=1, description="noise seed")
);
